<font size="5"><b><center>CS:GO Professional Matches Analysis</center></b></font>
<center><b>Dataset: </b>CS:GO Professional Matches - <b>Date: </b>March 26, 2020</center>

# Index
1. [Introduction](#Introduction)<br>
2. [Preparation](#Preparation)<br>
3. [Results DataFrame](#Results-DataFrame)<br>
3.1. [Distribution of scores](#Distribution-of-scores)<br>
3.2. [Most CT-sided map](#Most-CT-sided-map)<br>
3.3. [Maps played per period](#Maps-played-per-period)<br>
3.4. [Best teams on each map](#Best-teams-on-each-map)<br>
4. [Economy DataFrame](#Economy-DataFrame)<br>
4.1. [Round victory probability by equipment value](#Round-victory-probability-by-equipment-value)<br>
4.2. [Pistols ranking](#Pistols-ranking)<br>
5. [Players DataFrame](#Players-DataFrame)<br>
5.1.  [Players ranking by map](#Players-ranking-by-map)<br>
5.2. [Players ranking all maps](#Players-ranking-all-maps)<br>
5.3. [K/D Graph](#K/D-Graph)<br>
6. [Picks DataFrame](#Picks-DataFrame)<br>
7. [Further exploration and predictions](#Further-exploration-and-predictions)<br>

# Introduction

<b>Background:</b> The data used in this notebook was collected at https://www.hltv.org/results. It was scraped off using the libraries 'requests' and 'BeautifulSoup'.

I haven't set any specific goal for this analysis, the idea was just to explore the data and see what information I could get. Also, I tried to avoid computing statistics that are easily obtained in e-sports websites (team victory percentage, average k/d ratio per player, etc.) and instead focused on rankings and analysis with temporal aspects.

This is my first public kernel, so don't hesitate in giving me corrections and suggestions.

Let's get started!

# The Game

This section is copied from the Dataset description.

Counter-Strike Global Offensive is a game released in 2012, as a sequel to Counter-Strike Source (released in 2004), which is itself a sequel to the original Counter-Strike (released in 2000). The game's longevity is primarily caused by its competitive approach and vibrant professional scene. This longevity has shown in numbers recently, as CS:GO reached in March its all-time high concurrent weekly players (1.1M players), making it the most played game on Steam, 7 years after it was launched.

The game retains the same gameplay concepts since its first version, which include a Terrorist side (T) that is tasked to plant a bomb and have it detonate, and a Counter-Terrorist side (CT) that is tasked to defuse the bomb or prevent it from being planted. Both teams can also win a round by eliminating all players on the opposing team before the bomb is planted.

A standard game of Counter-Strike is a best of 30 rounds, the winning team being the first to win 16 rounds. The 30 rounds are played in two halves of 15 on each side of the map, with a round time limit of 1 minute 55 seconds, plus 40 seconds after the bomb is planted.

In case both teams draw at the 30th round on 15x15, 6 more rounds are added-on, which constitutes overtime. The overtime ends if a team wins 4 out of 6 rounds. If both teams win 3 rounds in overtime, another overtime of 6 rounds is played, and the process might repeat indefinitely until one team wins it.

There are 7 maps in the map pool that are available to be played competitively at any given time. Maps are removed and added frequently for updates and revamps, as to not make the game stale. Matches are normally played as a 'bo3' (Best of 3) maps, with less important matches played in a 'bo1' fashion and finals often played as 'bo5's.

There is a money management side to rounds in Counter-Strike. This side is detailed in the 'Economy DataFrame' topic.

# Preparation

Importing libraries and loading the tables.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from os import listdir

pd.set_option('display.max_columns',100)

listdir('../input/csgo-professional-matches/')

['picks.csv', 'players.csv', 'economy.csv', 'results.csv']

In [2]:
base_dir = '../input/csgo-professional-matches/'

results_df = pd.read_csv(base_dir+'results.csv',low_memory=False)
picks_df = pd.read_csv(base_dir+'picks.csv',low_memory=False)
economy_df = pd.read_csv(base_dir+'economy.csv',low_memory=False)
players_df = pd.read_csv(base_dir+'players.csv',low_memory=False)

The first rows of each table are presented below. The data is split into 4 tables that store data related to:

 - <b>results_df:</b>   &nbsp;  map scores and team rankings

 - <b>picks_df:</b> &nbsp;  order of map picks and vetos in the map selection process.

 - <b>economy_df:</b>  &nbsp;  round start equipment value for all rounds played

 - <b>players_df:</b> &nbsp;   individual performances of players in each map.
 
Values stored in 'event_id' and 'match_id' columns are unique for each match and event and shared between dataframes, so these columns can be used as keys to merge data between dataframes.

It is necessary to note that the rows in 'results_df' and 'economy_df' store data for each map played in a match, while the rows in 'picks_df' and 'players_df' store data for the entire match.

In [3]:
results_df.head()

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
0,2020-03-18,Recon 5,TeamOne,Dust2,0,16,2,2,0,1,0,15,5151,2340454,62,63,0,2,2
1,2020-03-18,Recon 5,TeamOne,Inferno,13,16,2,2,8,6,5,10,5151,2340454,62,63,0,2,2
2,2020-03-18,New England Whalers,Station7,Inferno,12,16,2,1,9,6,3,10,5243,2340461,140,118,12,16,2
3,2020-03-18,Rugratz,Bad News Bears,Inferno,7,16,2,2,0,8,7,8,5151,2340453,61,38,0,2,2
4,2020-03-18,Rugratz,Bad News Bears,Vertigo,8,16,2,2,4,5,4,11,5151,2340453,61,38,0,2,2


In [4]:
picks_df.head()

,date,team_1,team_2,inverted_teams,match_id,event_id,best_of,system,t1_removed_1,t1_removed_2,t1_removed_3,t2_removed_1,t2_removed_2,t2_removed_3,t1_picked_1,t2_picked_1,left_over
0,2020-03-18,TeamOne,Recon 5,1,2340454,5151,3,123412,Vertigo,Train,0.0,Nuke,Overpass,0.0,Dust2,Inferno,Mirage
1,2020-03-18,Rugratz,Bad News Bears,0,2340453,5151,3,123412,Dust2,Nuke,0.0,Mirage,Train,0.0,Vertigo,Inferno,Overpass
2,2020-03-18,New England Whalers,Station7,0,2340461,5243,1,121212,Mirage,Dust2,Vertigo,Nuke,Train,Overpass,0.0,0.0,Inferno
3,2020-03-17,Complexity,forZe,1,2340279,5226,3,123412,Inferno,Nuke,0.0,Overpass,Vertigo,0.0,Dust2,Train,Mirage
4,2020-03-17,Singularity,Endpoint,0,2340456,5247,3,123412,Train,Mirage,0.0,Nuke,Inferno,0.0,Overpass,Vertigo,Dust2


In [5]:
economy_df.head()

,date,match_id,event_id,team_1,team_2,best_of,_map,t1_start,t2_start,1_t1,2_t1,3_t1,4_t1,5_t1,6_t1,7_t1,8_t1,9_t1,10_t1,11_t1,12_t1,13_t1,14_t1,15_t1,16_t1,17_t1,18_t1,19_t1,20_t1,21_t1,22_t1,23_t1,24_t1,25_t1,26_t1,27_t1,28_t1,29_t1,30_t1,1_t2,2_t2,3_t2,4_t2,5_t2,6_t2,7_t2,8_t2,9_t2,10_t2,11_t2,12_t2,13_t2,14_t2,15_t2,16_t2,17_t2,18_t2,19_t2,20_t2,21_t2,22_t2,23_t2,24_t2,25_t2,26_t2,27_t2,28_t2,29_t2,30_t2,1_winner,2_winner,3_winner,4_winner,5_winner,6_winner,7_winner,8_winner,9_winner,10_winner,11_winner,12_winner,13_winner,14_winner,15_winner,16_winner,17_winner,18_winner,19_winner,20_winner,21_winner,22_winner,23_winner,24_winner,25_winner,26_winner,27_winner,28_winner,29_winner,30_winner
0,2020-03-01,2339402,4901,G2,Natus Vincere,5,Nuke,t,ct,4350.0,1100.0,22100.0,9350.0,25750.0,10400.0,24600.0,8150.0,26700.0,23400.0,4300.0,25900.0,11950.0,24850.0,21900.0,4150.0,10650.0,27300.0,27000.0,30950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4250.0,20250.0,24600.0,29300.0,30650.0,31450.0,32050.0,31450.0,32050.0,30950.0,29300.0,30200.0,31550.0,32350.0,33050.0,4250.0,20300.0,8250.0,1300.0,22200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-01,2339402,4901,G2,Natus Vincere,5,Dust2,ct,t,3900.0,7400.0,23250.0,28500.0,31900.0,31700.0,18950.0,30200.0,28650.0,30350.0,30150.0,11100.0,23700.0,8550.0,26350.0,4050.0,9400.0,21900.0,12700.0,11300.0,3100.0,26250.0,21300.0,23950.0,27450.0,27550.0,28050.0,26250.0,26250.0,NaN,3500.0,18550.0,7300.0,1500.0,21800.0,12000.0,21050.0,24450.0,6850.0,26850.0,23100.0,25650.0,26800.0,26750.0,28250.0,4000.0,18850.0,15850.0,23000.0,26850.0,29100.0,26300.0,26850.0,19050.0,3500.0,26450.0,27450.0,27500.0,29050.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,NaN
2,2020-03-01,2339402,4901,G2,Natus Vincere,5,Mirage,t,ct,4150.0,14300.0,2000.0,24800.0,9000.0,23150.0,21850.0,23700.0,10450.0,26250.0,8800.0,24950.0,12100.0,24350.0,18250.0,4300.0,19400.0,8900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4200.0,21200.0,24150.0,27050.0,31350.0,30650.0,31050.0,28150.0,29650.0,30950.0,31550.0,32950.0,32250.0,31650.0,33350.0,4350.0,14300.0,22850.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Dust2,t,ct,4150.0,18050.0,21000.0,25850.0,25000.0,25000.0,27250.0,26150.0,26300.0,27850.0,26750.0,27450.0,27850.0,18300.0,27850.0,4000.0,21100.0,9100.0,3100.0,22100.0,23350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4250.0,9300.0,2100.0,19300.0,27250.0,9850.0,24800.0,12200.0,27550.0,26100.0,7450.0,27650.0,24500.0,19100.0,18050.0,4150.0,15100.0,18450.0,23150.0,28150.0,27850.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Nuke,ct,t,4200.0,10000.0,22000.0,24500.0,27550.0,29350.0,31950.0,31850.0,31750.0,32850.0,32150.0,31750.0,31850.0,33050.0,33150.0,4250.0,3000.0,21150.0,11750.0,28050.0,25900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4200.0,17950.0,9050.0,1000.0,22850.0,7500.0,24400.0,24700.0,9700.0,27150.0,21850.0,21900.0,8150.0,25200.0,20450.0,4300.0,19300.0,27400.0,31700.0,29650.0,12150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
players_df.head()

,date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,map_1,map_2,map_3,kills,assists,deaths,hs,flash_assists,kast,kddiff,adr,fkdiff,rating,m1_kills,m1_assists,m1_deaths,m1_hs,m1_flash_assists,m1_kast,m1_kddiff,m1_adr,m1_fkdiff,m1_rating,m2_kills,m2_assists,m2_deaths,m2_hs,m2_flash_assists,m2_kast,m2_kddiff,m2_adr,m2_fkdiff,m2_rating,m3_kills,m3_assists,m3_deaths,m3_hs,m3_flash_assists,m3_kast,m3_kddiff,...,m3_fkdiff,m3_rating,kills_ct,deaths_ct,kddiff_ct,adr_ct,kast_ct,rating_ct,kills_t,deaths_t,kddiff_t,adr_t,kast_t,rating_t,m1_kills_ct,m1_deaths_ct,m1_kddiff_ct,m1_adr_ct,m1_kast_ct,m1_rating_ct,m1_kills_t,m1_deaths_t,m1_kddiff_t,m1_adr_t,m1_kast_t,m1_rating_t,m2_kills_ct,m2_deaths_ct,m2_kddiff_ct,m2_adr_ct,m2_kast_ct,m2_rating_ct,m2_kills_t,m2_deaths_t,m2_kddiff_t,m2_adr_t,m2_kast_t,m2_rating_t,m3_kills_ct,m3_deaths_ct,m3_kddiff_ct,m3_adr_ct,m3_kast_ct,m3_rating_ct,m3_kills_t,m3_deaths_t,m3_kddiff_t,m3_adr_t,m3_kast_t,m3_rating_t
0,2020-02-26,Brehze,Evil Geniuses,Liquid,United States,9136,2339385,4901,IEM Katowice 2020,3,Overpass,Nuke,Inferno,57,14,61,29,0.0,71.1,-4,79.9,0,1.04,11,3,18,5,0.0,65.2,-7,60.8,-1,0.70,30.0,7.0,24.0,16.0,0.0,73.5,6.0,99.2,6.0,1.38,16.0,4.0,19.0,8.0,0.0,73.1,-3.0,...,-5.0,0.91,34.0,30.0,4.0,81.6,79.2,1.10,23.0,31.0,-8.0,77.5,60.0,0.97,8.0,10.0,-2.0,76.3,73.3,0.90,3.0,8.0,-5.0,31.9,50.0,0.34,17.0,10.0,7.0,93.7,83.3,1.41,13.0,14.0,-1.0,105.3,62.5,1.35,9.0,10.0,-1.0,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89
1,2020-02-26,CeRq,Evil Geniuses,Liquid,Bulgaria,11219,2339385,4901,IEM Katowice 2020,3,Overpass,Nuke,Inferno,54,10,54,18,4.0,65.1,0,71.7,2,0.98,11,2,17,4,2.0,60.9,-6,68.9,-1,0.75,26.0,6.0,19.0,8.0,1.0,76.5,7.0,80.1,3.0,1.24,17.0,2.0,18.0,6.0,1.0,53.8,-1.0,...,0.0,0.87,37.0,25.0,12.0,77.4,72.9,1.16,17.0,29.0,-12.0,63.9,54.3,0.73,9.0,9.0,0.0,72.3,73.3,0.88,2.0,8.0,-6.0,62.4,37.5,0.50,15.0,6.0,9.0,79.8,88.9,1.45,11.0,13.0,-2.0,80.5,62.5,1.00,13.0,10.0,3.0,79.5,53.3,1.12,4.0,8.0,-4.0,40.7,54.5,0.53
2,2020-02-26,EliGE,Liquid,Evil Geniuses,United States,8738,2339385,4901,IEM Katowice 2020,3,Overpass,Nuke,Inferno,55,10,51,28,1.0,67.5,4,77.9,1,1.08,15,3,12,9,0.0,69.6,3,77.0,3,1.32,24.0,3.0,24.0,14.0,0.0,64.7,0.0,72.9,-1.0,0.97,16.0,4.0,15.0,5.0,1.0,69.2,1.0,...,-1.0,1.04,31.0,17.0,14.0,96.6,71.4,1.39,24.0,34.0,-10.0,64.2,64.6,0.86,11.0,2.0,9.0,135.2,75.0,2.17,4.0,10.0,-6.0,45.9,66.7,0.87,13.0,9.0,4.0,87.6,75.0,1.26,11.0,15.0,-4.0,59.7,55.6,0.71,7.0,6.0,1.0,81.5,63.6,1.03,9.0,9.0,0.0,87.9,73.3,1.05
3,2020-02-26,Ethan,Evil Geniuses,Liquid,United States,10671,2339385,4901,IEM Katowice 2020,3,Overpass,Nuke,Inferno,43,5,54,18,2.0,65.1,-11,58.7,-4,0.83,11,1,15,6,1.0,65.2,-4,60.7,-2,0.73,22.0,3.0,21.0,8.0,1.0,70.6,1.0,67.9,-2.0,1.00,10.0,1.0,18.0,4.0,0.0,57.7,-8.0,...,0.0,0.69,33.0,23.0,10.0,74.0,75.0,1.11,10.0,31.0,-21.0,37.8,51.4,0.43,9.0,8.0,1.0,68.3,73.3,0.92,2.0,7.0,-5.0,46.5,50.0,0.38,15.0,6.0,9.0,84.3,83.3,1.40,7.0,15.0,-8.0,49.3,56.2,0.55,9.0,9.0,0.0,67.2,66.7,0.97,1.0,9.0,-8.0,14.8,45.5,0.31
4,2020-02-26,NAF,Liquid,Evil Geniuses,Canada,8520,2339385,4901,IEM Katowice 2020,3,Overpass,Nuke,Inferno,52,22,46,23,9.0,77.1,6,75.9,-1,1.08,10,5,12,3,3.0,65.2,-2,51.5,0,0.83,29.0,6.0,21.0,16.0,0.0,82.4,8.0,101.9,0.0,1.35,13.0,11.0,13.0,4.0,6.0,80.8,0.0,...,-1.0,0.98,28.0,17.0,11.0,96.3,85.7,1.36,24.0,29.0,-5.0,61.0,70.8,0.87,4.0,3.0,1.0,64.8,62.5,0.98,6.0,9.0,-3.0,44.4,66.7,0.75,19.0,8.0,11.0,128.1,100.0,1.88,10.0,13.0,-3.0,78.7,66.7,0.89,5.0,6.0,-1.0,72.9,81.8,0.96,8.0,7.0,1.0,56.3,80.0,0.99


The data collected has data from all professional CS:GO matches, including matches from relatively unknown teams. For that reason, we will be limiting the datasets to the matches played between the top 30 teams in the HLTV rankings.

In [7]:
min_rank = 30
results_df = results_df[(results_df.rank_1<min_rank)&(results_df.rank_2<min_rank)]

picks_df     = picks_df  [picks_df  .match_id.isin(results_df.match_id.unique())]
economy_df   = economy_df[economy_df.match_id.isin(results_df.match_id.unique())]
players_df   = players_df[players_df.match_id.isin(results_df.match_id.unique())]

# Results DataFrame

## Distribution of scores

In [8]:
winner_1 = results_df[results_df.result_1>=results_df.result_2].result_1.values
loser_1  = results_df[results_df.result_1>=results_df.result_2].result_2.values

winner_2 = results_df[results_df.result_1<results_df.result_2].result_2.values
loser_2  = results_df[results_df.result_1<results_df.result_2].result_1.values

winner = np.concatenate((winner_1,winner_2))
loser = np.concatenate((loser_1,loser_2))
scores_df = pd.DataFrame(np.vstack((winner,loser)).T,columns=['winner','loser'])

In [9]:
gb = scores_df.groupby(by=['winner','loser'])['winner'].count()/scores_df.shape[0]
overtime_percentage = str(round(gb[gb.index.get_level_values(0)!=16].sum()*100,1))+'%'

gb = round(gb[gb>10**-3]*100,1)

index_plot = np.array(gb.index.get_level_values(0).astype('str'))+'-'+np.array(
    gb.index.get_level_values(1).astype('str'))

fig = go.Figure()
fig.add_trace(go.Scatter(x=index_plot,y=gb.values, name='results'))
fig.update_layout(xaxis_type='category',title='Scores distribution',xaxis_title='Score',yaxis_title='Percentage of matches (%)')

In [10]:
overtime_percentage

'9.7%'

We can see that on regular time (disregarding overtime), the most common score is 16-14 (achieved in 10.7% of the matches) and the rarest score is 16-0 (achieved in only 0.2% of the matches), with intermediate scores falling somewhere in between. 

9.7% of the matches go to overtime.

The results may differ if we consider matches played by non top-tier teams.

## Most CT sided map

There has long been a dispute in CS:GO to determine the most CT-sided maps, and always present discussion if having a heavily one-sided map is a desirable outcome. Here we determine this characteristic by computing the average scores obtained in each side of the map and then comparing both sides.

In [11]:
ct_1 = results_df[['date','_map','ct_1']].rename(columns={'ct_1':'ct'})
ct_2 = results_df[['date','_map','ct_2']].rename(columns={'ct_2':'ct'})
ct = pd.concat((ct_1,ct_2))

In [12]:
t_1 = results_df[['date','_map','t_1']].rename(columns={'t_1':'t'})
t_2 = results_df[['date','_map','t_2']].rename(columns={'t_2':'t'})
t = pd.concat((t_1,t_2))

In [13]:
t = t.sort_values('date')
ct = ct.sort_values('date')

In [14]:
maps = ['Cache','Cobblestone','Dust2','Inferno','Mirage','Nuke','Overpass','Train','Vertigo']

In [15]:
series_t, series_ct, how_ct = {},{},{}
for i, key in enumerate(maps):
    t_map = t[t._map == maps[i]]
    ct_map = ct[ct._map == maps[i]]
    y_t = t_map.t.rolling(min_periods = 20, window= 200, center=True).sum().values
    y_ct = ct_map.ct.rolling(min_periods = 20, window= 200, center=True).sum().values
    
    series_t[key] = pd.Series(data=y_t,index=t_map.date)
    series_ct[key] = pd.Series(data=y_ct,index=ct_map.date)
    
    how_ct[key] = series_ct[key]/(series_ct[key]+series_t[key])//0.001/10

In [16]:
def add_trace(_map):
    fig.add_trace(go.Scatter(x=how_ct[_map].index, y=how_ct[_map].values, name=_map))

In [17]:
fig = go.Figure()
for _map in maps:
    add_trace(_map)
fig.add_trace(go.Scatter(x=['2015-11-01', '2020-03-12'], y=[50,50],
                         mode='lines',line=dict(color='grey'),showlegend=False))
fig.update_layout(title='Distribution of rounds between CT and T sides',
                  yaxis_title='Percentage of round won on the CT-side (%)')
fig.show()

There are long stretches without data for a map in the graph. This happens because maps are added and removed from the map pool constantly.

<b>Nuke</b> and <b>Train</b> oscilatte as being the most CT-sided maps, having around 57% of the rounds won on the CT-side, while
<b>Dust2</b> and <b>Cache</b> are historically the most T-sided maps.

It is interesting to note that <b>Inferno</b> was known for being a heavily CT-sided map prior to 2016, which was one of the reasons to update it. Since its update, <b>Inferno</b> has actually been the most balanced map in this aspect.



## Maps played per period

About the maps:
 - <b>Mirage</b>, <b>Train</b>, <b>Inferno</b> and <b>Overpass</b> are the maps from which we have the most data available. They are also the maps present on the map pool for the longest time;

 - <b>Cache</b>, <b>Cobblestone</b> and <b>Dust2</b> have been played less, but have also been outside the map pool for the longest periods:

 - <b>Nuke</b> is historically the least played map, even though it has been present in the map pool for a long time. The only explanation for this stems from the teams' unfamiliarity with the map;

 - <b>Vertigo</b> has limited data available as it was the most recently added map in the map pool.

In [18]:
print('Total number of matches played on the map:')
results_df.groupby('_map').date.count()

Total number of matches played on the map:


_map
Cache           900
Cobblestone     898
Default           2
Dust2           892
Inferno        1325
Mirage         1617
Nuke            750
Overpass       1150
Train          1384
Vertigo          99
Name: date, dtype: int64

In CS:GO, the most reputable tournaments are the Majors. These tournaments are normally played twice a year and have a prize pool of $1,000,000. More information about the subject can be seen here: https://liquipedia.net/counterstrike/Majors

For the next step, we are going to discretize the 'date' column in a dataframe into a 'time_period' column. This new column will refer to the most recently played major tournament.

As an example, following this binning technique, we are currently on the Berlin 2019 period, as that was the most recently played tournament.

In [19]:
majors = [{'tournament':'01. Cluj-Napoca 2015','start_date':'2015-10-28'},
          {'tournament':'02. Columbus 2016','start_date':'2016-03-29'},
          {'tournament':'03. Cologne 2016','start_date':'2016-07-05'},
          {'tournament':'04. Atlanta 2017','start_date':'2017-01-22'},
          {'tournament':'05. Krakow 2017','start_date':'2017-07-16'},
          {'tournament':'06. Boston 2018','start_date':'2018-01-26'},
          {'tournament':'07. London 2018','start_date':'2018-09-20'},
          {'tournament':'08. Katowice 2019','start_date':'2019-02-28'},
          {'tournament':'09. Berlin 2019','start_date':'2019-09-05'}]

In [20]:
def create_col_time_period(df):
    df['time_period'] = ''
    
    for major_start in majors:
        df.loc[(df['date']>=major_start['start_date']),'time_period'] = major_start['tournament']
    
    return df

In [21]:
results_df = create_col_time_period(results_df)
economy_df = create_col_time_period(economy_df)
picks_df = create_col_time_period(picks_df)
players_df = players_df.merge(results_df[['match_id','time_period']],'left',on='match_id')

In [22]:
results_df_team_1 = results_df[['time_period','team_1','_map','ct_1','t_2','ct_2','t_1']
                      ].rename(columns={'team_1':'team'})
results_df_team_2 = results_df[['time_period','team_2','_map','ct_1','t_2','ct_2','t_1']
                      ].rename(columns={'team_2':'team'})
results_df_teams = pd.concat((results_df_team_1,results_df_team_2))[['time_period','team','_map']]

In [23]:
gb = results_df_teams.groupby(['time_period','_map']).team.count()
gb_text = round(gb*100/gb.groupby('time_period').sum(),1).reset_index().rename(columns={'team':'percentage'})
gb_text.percentage = gb_text.percentage.astype(str)+'%'
gb = gb.reset_index()

In [24]:
fig = go.Figure()
for _map in maps:
    fig.add_bar(name=_map,x=gb[gb._map==_map].time_period,y=gb[gb._map==_map].team,
                text=gb_text[gb_text._map==_map].percentage,textposition='inside')

fig.update_layout(barmode='stack',legend=dict(traceorder='normal'),yaxis_title='Number of maps played',font=dict(size=10))
fig.show()

As pointed out previously, <b>Nuke</b> is historically the least popular map in the pool. This has been changing recently, as teams that used to permaban the map have moved on to banning maps like Vertigo.

<b>Vertigo</b>, as the newest and most unconventional map, is also the most unpopular map, probably due to the many changes it has had in its short competitive term.

The period between Columbus and Cologne 2016 has the least amount of maps played and is also the shortest (under 4 months), while the period between Boston and London 2018 has the highest amount of maps played and is also the longest (over 7 months).

## Best teams on each map

In this section, let's search for the best teams in each map. The victory percentage for the teams on each map is readily available in many e-sports sites like hltv.org, so it would be pointless to present the information the same way. For this reason, instead of pointing out the percentages, we are going to rank the best teams on each map and each side (CT and T) for every time period.

In [25]:
results_df_team_1_ct = results_df_team_1.rename(columns={'ct_1':'ct_team','t_2':'t_opponent'}).drop(columns=['ct_2','t_1'])
results_df_team_2_ct = results_df_team_2.rename(columns={'ct_2':'ct_team','t_1':'t_opponent'}).drop(columns=['ct_1','t_2'])
results_df_ct = pd.concat((results_df_team_1_ct,results_df_team_2_ct),sort=True)

results_df_team_1_t = results_df_team_1.rename(columns={'t_1':'t_team','ct_2':'ct_opponent'}).drop(columns=['ct_1','t_2'])
results_df_team_2_t = results_df_team_2.rename(columns={'t_2':'t_team','ct_1':'ct_opponent'}).drop(columns=['ct_2','t_1'])
results_df_t = pd.concat((results_df_team_1_t,results_df_team_2_t),sort=True)

In [26]:
results_df_ct['side_diff'] = results_df_ct['ct_team']-results_df_ct['t_opponent']
results_df_ct['side_sum'] = results_df_ct['ct_team']+results_df_ct['t_opponent']

results_df_t['side_diff'] = results_df_t['t_team']-results_df_t['ct_opponent']
results_df_t['side_sum']  = results_df_t['t_team'] +results_df_t['ct_opponent']

results_df_ct.head()

,_map,ct_team,t_opponent,team,time_period,side_diff,side_sum
9,Mirage,9,6,fnatic,09. Berlin 2019,3,15
21,Overpass,4,4,fnatic,09. Berlin 2019,0,8
22,Nuke,7,8,Natus Vincere,09. Berlin 2019,-1,15
23,Dust2,5,10,Natus Vincere,09. Berlin 2019,-5,15
24,Train,3,7,forZe,09. Berlin 2019,-4,10


In [27]:
def groupby_time_map_team(results_df_side):
    gb = results_df_side.groupby(['time_period','_map','team'])['side_diff','side_sum'].sum()
    gb['side_diff_per_game'] = gb['side_diff']/(gb['side_sum']/15)
    gb = gb.sort_values(['time_period','_map','side_diff_per_game'],ascending=[1,1,0])

    for major in majors:
        col = major['tournament']
        _filter = (gb.side_sum > gb.loc[col].side_sum.mean()*3/4)
        gb.loc[col] = gb.loc[_filter][gb.loc[_filter].index.get_level_values(0)==col]

    gb.dropna(inplace=True)    

    return gb

In [28]:
gb_ct = groupby_time_map_team(results_df_ct)
gb_t = groupby_time_map_team(results_df_t)

In [29]:
def plot_ranking_teams_sides(gb):
    rankings_teams = {}
    for _map in maps:
        rankings_teams[_map] = pd.DataFrame(index=range(1,6),)
        rankings_teams[_map].index.name = 'ranking'
        rankings_teams[_map].style.set_caption(_map)

        for major in majors:
            col = major['tournament']
            try:
                rankings_teams[_map][col] = gb.loc[col,_map]['side_diff_per_game'][:5].index
            except:
                pass
        print('\n'+_map+':')
        display(rankings_teams[_map])

In [30]:
print('T-side Rankings:\n')
plot_ranking_teams_sides(gb_t)

T-side Rankings:


Cache:


,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018
ranking,,,,,,,
1,NiP,mousesports,Envy,FaZe,Renegades,Liquid,NiP
2,fnatic,Astralis,Cloud9,North,Luminosity,FaZe,Liquid
3,Splyce,Renegades,mousesports,fnatic,Space Soldiers,TYLOO,Virtus.pro
4,Luminosity,Liquid,GODSENT,SK,SK,Gambit,AVANGAR
5,Liquid,OpTic,G2,G2,G2,SK,NRG



Cobblestone:


,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018
ranking,,,,,,
1,Tempo Storm,Liquid,VG.CyberZen,SK,SK,Space Soldiers
2,HellRaisers,Envy,X,BIG,Space Soldiers,G2
3,Virtus.pro,fnatic,Virtus.pro,Gambit,Cloud9,HellRaisers
4,OpTic,GODSENT,Gambit,G2,NiP,GODSENT
5,NiP,Luminosity,Envy,Cloud9,North,fnatic



Dust2:


...

,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018,08. Katowice 2019,09. Berlin 2019
ranking,,,,,,,,,
1,TSM,G2,Natus Vincere,Astralis,SK,GODSENT,Astralis,Liquid,FURIA
2,?,OpTic,NiP,FlipSid3,Space Soldiers,NRG,NRG,Heroic,fnatic
3,Astralis,NiP,OpTic,Kinguin,FaZe,Astralis,ENCE,OpTic,Astralis
4,fnatic,Astralis,FaZe,Heroic,OpTic,NiP,Natus Vincere,MIBR,Tricked
5,HellRaisers,HellRaisers,Cloud9,Gambit,North,North,Cloud9,Vitality,NiP



Train:


,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018,08. Katowice 2019,09. Berlin 2019
ranking,,,,,,,,,
1,NiP,FaZe,Cloud9,BIG,Natus Vincere,Natus Vincere,Renegades,Natus Vincere,Sprout
2,fnatic,fnatic,SK,Natus Vincere,FlipSid3,NRG,Astralis,NRG,FURIA
3,Luminosity,Liquid,OpTic,Liquid,NiP,Windigo,North,Valiance,NiP
4,Vexed,Astralis,X,Gambit,OpTic,GODSENT,mousesports,fnatic,Astralis
5,Natus Vincere,NiP,Astralis,FaZe,Cloud9,mousesports,Natus Vincere,North,mousesports



Vertigo:


,09. Berlin 2019
ranking,
1,GODSENT
2,G2
3,MIBR
4,NiP
5,Heroic


In [31]:
print('CT-side Rankings:\n')
plot_ranking_teams_sides(gb_ct)

CT-side Rankings:


Cache:


,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018
ranking,,,,,,,
1,Enemy,G2,fnatic,mousesports,Space Soldiers,Astralis,AVANGAR
2,fnatic,Tempo Storm,Astralis,Cloud9,Gambit,HellRaisers,BIG
3,Cloud9,SK,Epsilon,NiP,NiP,FaZe,Liquid
4,Envy,HellRaisers,HellRaisers,SK,FaZe,AVANGAR,Renegades
5,G2,mousesports,Cloud9,Immortals,HellRaisers,G2,Cloud9



Cobblestone:


,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018
ranking,,,,,,
1,fnatic,Cloud9,X,G2,mousesports,Liquid
2,FlipSid3,Envy,Immortals,SK,Virtus.pro,AGO
3,Luminosity,Renegades,Epsilon,Virtus.pro,fnatic,Space Soldiers
4,HellRaisers,NiP,OpTic,Renegades,Natus Vincere,fnatic
5,SK,HellRaisers,SK,North,Renegades,GODSENT



Dust2:


...

,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018,08. Katowice 2019,09. Berlin 2019
ranking,,,,,,,,,
1,mousesports,Luminosity,Natus Vincere,BIG,FaZe,NRG,ENCE,North,Natus Vincere
2,?,G2,GODSENT,SK,SK,Astralis,FaZe,Liquid,Sprout
3,Luminosity,Astralis,OpTic,North,NiP,BIG,LDLC,NiP,mousesports
4,fnatic,FlipSid3,mousesports,HellRaisers,Astralis,FaZe,Liquid,Astralis,FURIA
5,NiP,CLG,SK,Gambit,G2,G2,Astralis,Vitality,Tricked



Train:


,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018,08. Katowice 2019,09. Berlin 2019
ranking,,,,,,,,,
1,Gambit,Orbit,SK,Natus Vincere,AGO,Astralis,ENCE,NRG,AVANGAR
2,Luminosity,Tempo Storm,Gambit,NiP,fnatic,Imperial,FaZe,Astralis,CR4ZY
3,Cloud9,NRG,fnatic,mousesports,Liquid,Cloud9,mousesports,ENCE,mousesports
4,?,Luminosity,GODSENT,SK,SK,FaZe,Astralis,Natus Vincere,Astralis
5,Liquid,Natus Vincere,FaZe,Astralis,Cloud9,HellRaisers,Natus Vincere,FaZe,Natus Vincere



Vertigo:


,09. Berlin 2019
ranking,
1,Astralis
2,G2
3,Heroic
4,Liquid
5,NiP


# Economy DataFrame

## Round victory probability by equipment value

Counter-strike has an economic system that governs the acquisitions of armor, weapons and grenades by the players. The rules of this system have changed many times in the past. 

Currently, round loss bonuses are based on a count of a team's round loss bonus. This count is increased by one after every loss and decreased by one after every win (minimum 0). The money returned after losing a round is:

 - 0 Losses: &dollar;1400;
 - 1 Loss: &dollar;1900;
 - 2 Losses: &dollar;2400;
 - 3 Losses: &dollar;2900;
 - 4+ Losses: &dollar;3400.

Previously to 2019, a win would reset the round loss bonus back to &dollar;1400 per player. Now, a win reduces the loss count by one.
Teams begin the half with a loss count of 1, so that losing the pistol round is not as damaging to the economy.

Players start the half with &dollar;800, which is enough to buy either armour or an improved pistol, but not both, nor better weapons. That is why rounds 1 and 16 are called pistol rounds.

Winning a round by eliminations grants &dollar;3250 to the winning team. If the win was achieved by defusing the bomb as CT or by having it detonate as T, the reward is of 3500 dollars.

Another way of getting money in the match is by killing players in the opposing team. The money received per kill varies according to the weapon wielded:
- &dollar;100: Sniper rifle kills;
- &dollar;300: pistol, assault rifle and grenade kills;
- &dollar;600: SMG kills;
- &dollar;900: Shotgun kills;
- &dollar;1500: Knife kills.

In the HLTV economy section of the matches, the sum of the equipment value of the teams is categorized according to 4 equipment value ranges:
 - &dollar; 0-5 K: Eco;
 - &dollar; 5-10 K: Forced Pistols;
 - &dollar; 10-20 K: Semi-buy;
 - &dollar; 20+ K: Full buy.
 
In this notebooks, the Semi-buy category was split into two new categories, according to equipment value:
 - &dollar;10-15 K: ForcedSMGs;
 - &dollar;15-20 K: ForcedBuy.
 
This change was added because the &dollar;10-20 K money range is too broad and groups together very distinct sets of equipment.

In [32]:
economy_df.head()

,date,match_id,event_id,team_1,team_2,best_of,_map,t1_start,t2_start,1_t1,2_t1,3_t1,4_t1,5_t1,6_t1,7_t1,8_t1,9_t1,10_t1,11_t1,12_t1,13_t1,14_t1,15_t1,16_t1,17_t1,18_t1,19_t1,20_t1,21_t1,22_t1,23_t1,24_t1,25_t1,26_t1,27_t1,28_t1,29_t1,30_t1,1_t2,2_t2,3_t2,4_t2,5_t2,6_t2,7_t2,8_t2,9_t2,10_t2,11_t2,12_t2,13_t2,14_t2,15_t2,16_t2,17_t2,18_t2,19_t2,20_t2,21_t2,22_t2,23_t2,24_t2,25_t2,26_t2,27_t2,28_t2,29_t2,30_t2,1_winner,2_winner,3_winner,4_winner,5_winner,6_winner,7_winner,8_winner,9_winner,10_winner,11_winner,12_winner,13_winner,14_winner,15_winner,16_winner,17_winner,18_winner,19_winner,20_winner,21_winner,22_winner,23_winner,24_winner,25_winner,26_winner,27_winner,28_winner,29_winner,30_winner,time_period
0,2020-03-01,2339402,4901,G2,Natus Vincere,5,Nuke,t,ct,4350.0,1100.0,22100.0,9350.0,25750.0,10400.0,24600.0,8150.0,26700.0,23400.0,4300.0,25900.0,11950.0,24850.0,21900.0,4150.0,10650.0,27300.0,27000.0,30950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4250.0,20250.0,24600.0,29300.0,30650.0,31450.0,32050.0,31450.0,32050.0,30950.0,29300.0,30200.0,31550.0,32350.0,33050.0,4250.0,20300.0,8250.0,1300.0,22200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09. Berlin 2019
1,2020-03-01,2339402,4901,G2,Natus Vincere,5,Dust2,ct,t,3900.0,7400.0,23250.0,28500.0,31900.0,31700.0,18950.0,30200.0,28650.0,30350.0,30150.0,11100.0,23700.0,8550.0,26350.0,4050.0,9400.0,21900.0,12700.0,11300.0,3100.0,26250.0,21300.0,23950.0,27450.0,27550.0,28050.0,26250.0,26250.0,NaN,3500.0,18550.0,7300.0,1500.0,21800.0,12000.0,21050.0,24450.0,6850.0,26850.0,23100.0,25650.0,26800.0,26750.0,28250.0,4000.0,18850.0,15850.0,23000.0,26850.0,29100.0,26300.0,26850.0,19050.0,3500.0,26450.0,27450.0,27500.0,29050.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,NaN,09. Berlin 2019
2,2020-03-01,2339402,4901,G2,Natus Vincere,5,Mirage,t,ct,4150.0,14300.0,2000.0,24800.0,9000.0,23150.0,21850.0,23700.0,10450.0,26250.0,8800.0,24950.0,12100.0,24350.0,18250.0,4300.0,19400.0,8900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4200.0,21200.0,24150.0,27050.0,31350.0,30650.0,31050.0,28150.0,29650.0,30950.0,31550.0,32950.0,32250.0,31650.0,33350.0,4350.0,14300.0,22850.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09. Berlin 2019
3,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Dust2,t,ct,4150.0,18050.0,21000.0,25850.0,25000.0,25000.0,27250.0,26150.0,26300.0,27850.0,26750.0,27450.0,27850.0,18300.0,27850.0,4000.0,21100.0,9100.0,3100.0,22100.0,23350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4250.0,9300.0,2100.0,19300.0,27250.0,9850.0,24800.0,12200.0,27550.0,26100.0,7450.0,27650.0,24500.0,19100.0,18050.0,4150.0,15100.0,18450.0,23150.0,28150.0,27850.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09. Berlin 2019
4,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Nuke,ct,t,4200.0,10000.0,22000.0,24500.0,27550.0,29350.0,31950.0,31850.0,31750.0,32850.0,32150.0,31750.0,31850.0,33050.0,33150.0,4250.0,3000.0,21150.0,11750.0,28050.0,25900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4200.0,17950.0,9050.0,1000.0,22850.0,7500.0,24400.0,24700.0,9700.0,27150.0,21850.0,21900.0,8150.0,25200.0,20450.0,4300.0,19300.0,27400.0,31700.0,29650.0,12150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09. Berlin 2019


In [33]:
money_columns = ['2_t1','3_t1','4_t1','5_t1','6_t1','7_t1','8_t1','9_t1','10_t1','11_t1','12_t1','13_t1','14_t1'
                ,'15_t1','17_t1','18_t1','19_t1','20_t1','21_t1','22_t1','23_t1','24_t1','25_t1','26_t1','27_t1',
                 '28_t1','29_t1','30_t1',
                '2_t2','3_t2','4_t2','5_t2','6_t2','7_t2','8_t2','9_t2','10_t2','11_t2','12_t2','13_t2','14_t2'
                ,'15_t2','17_t2','18_t2','19_t2','20_t2','21_t2','22_t2','23_t2','24_t2','25_t2','26_t2','27_t2',
                 '28_t2','29_t2','30_t2']

economy_categories = {0:{'name':'eco','start':0,'end':5000},
                      1:{'name':'forcedPistols','start':5000,'end':10000},
                      2:{'name':'forcedSMGs','start':10000,'end':15000},
                      3:{'name':'forcedBuy','start':15000,'end':20000},
                      4:{'name':'fullBuy','start':20000,'end':50000}
                      }

In [34]:
for col in money_columns:
    for key, category in economy_categories.items():
        economy_df.loc[(economy_df[col]>category['start']) & (economy_df[col]<=category['end']),col] = key
    for key, category in economy_categories.items():
        economy_df.loc[economy_df[col]==key,col] = category['name']

In [35]:
def get_economy_stats(category):

    wins_by_side_t1 = pd.DataFrame([[0,0,0],[0,0,0]],index=['ct','t'],columns=['sum','count','mean'])
    wins_by_side_t2 = pd.DataFrame([[0,0,0],[0,0,0]],index=['ct','t'],columns=['sum','count','mean'])

    for _round in range(2,16):
        gb_1 = economy_df[economy_df[str(_round)+'_t1']==category].rename(columns={'t1_start':'side'}).groupby('side')[str(_round)+'_winner']
        gb_1 = gb_1.agg(['sum','count','mean'])

        gb_3 = economy_df[economy_df[str(_round+15)+'_t1']==category].rename(columns={'t2_start':'side'}).groupby('side')[str(_round+15)+'_winner']
        gb_3 = gb_3.agg(['sum','count','mean'])

        gb_1 = gb_1.reindex(['ct','t'], fill_value=0)
        gb_3 = gb_3.reindex(['ct','t'], fill_value=0)

        wins_by_side_t1 = wins_by_side_t1 + gb_1 + gb_3

    wins_by_side_t1['sum'] = 2*wins_by_side_t1['count']-wins_by_side_t1['sum']

    for _round in range(2,16):
        gb_2 = economy_df[economy_df[str(_round)+'_t2']==category].rename(columns={'t2_start':'side'}).groupby('side')[str(_round)+'_winner']
        gb_2 = gb_2.agg(['sum','count','mean'])

        gb_4 = economy_df[economy_df[str(_round+15)+'_t2']==category].rename(columns={'t1_start':'side'}).groupby('side')[str(_round+15)+'_winner']
        gb_4 = gb_4.agg(['sum','count','mean'])

        gb_2 = gb_2.reindex(['ct','t'], fill_value=0)
        gb_4 = gb_4.reindex(['ct','t'], fill_value=0)

        wins_by_side_t2 = wins_by_side_t2 + gb_2 + gb_4

    wins_by_side_t2['sum'] = wins_by_side_t2['sum']-wins_by_side_t2['count']

    wins_by_side = wins_by_side_t1 + wins_by_side_t2

    wins_by_side['mean'] = wins_by_side['sum']/wins_by_side['count']//0.001/10
    wins_by_side['num_per_game'] = wins_by_side['count']/economy_df.shape[0]//0.1/10
    wins_by_side = wins_by_side[['mean','num_per_game']]
    
    return wins_by_side

In [36]:
economy_stats = {}
mean_victories_df = pd.DataFrame(index=['ct','t'])
num_per_game_df = pd.DataFrame(index=['ct','t'])

for category in economy_categories.values():
    cat = category['name']
    economy_stats[cat] = get_economy_stats(cat)
    mean_victories_df[cat] = economy_stats[cat]['mean']
    num_per_game_df[cat] = economy_stats[cat]['num_per_game']

print('\nVictory probability (%):')
display(mean_victories_df)
print('\nNumber per game:')
display(num_per_game_df)


Victory probability (%):


,eco,forcedPistols,forcedSMGs,forcedBuy,fullBuy
ct,5.9,19.3,30.5,54.9,62.4
t,4.3,18.4,31.9,60.4,56.4



Number per game:


,eco,forcedPistols,forcedSMGs,forcedBuy,fullBuy
ct,2.1,1.9,1.1,1.5,16.9
t,2.1,1.8,1.0,2.6,16.0


The tables above present the probability of winning rounds according to the team's equipment value category, and the number of rounds of each category type in a map. The results do not take into consideration the equipment category of the opposing team.

Teams are 40% more likely to win <b>eco</b> rounds on the CT-side, 10% more likely to win <b>forcedBuy</b> rounds on the T-side and 11% more likely to win <b>fullBuy</b> rounds on the CT-side.

A team has on average 6 <b>non-fullBuy</b> rounds in a map. The number of rounds in each category does not vary much between CT and T sides, which is surprising, as the CT-side economy usually seems to be the hardest one to maintain. 

## Pistols ranking

In [37]:
gb_team_1_first_pistol   = economy_df.rename(columns={'team_1':'team','t1_start':'side'}).groupby(['side','time_period','team'])['1_winner'].agg(['mean','count'])
gb_team_1_second_pistol  = economy_df.rename(columns={'team_1':'team','t2_start':'side'}).groupby(['side','time_period','team'])['16_winner'].agg(['mean','count'])

gb_team_2_first_pistol   = economy_df.rename(columns={'team_2':'team','t2_start':'side'}).groupby(['side','time_period','team'])['1_winner'].agg(['mean','count'])
gb_team_2_second_pistol  = economy_df.rename(columns={'team_2':'team','t1_start':'side'}).groupby(['side','time_period','team'])['16_winner'].agg(['mean','count'])

In [38]:
gb = (2-gb_team_1_first_pistol['mean'])*gb_team_1_first_pistol['count']+(
    2-gb_team_1_second_pistol['mean'])*gb_team_1_second_pistol['count']+(
    gb_team_2_first_pistol['mean']-1)*gb_team_2_first_pistol['count']+(
    gb_team_2_second_pistol['mean']-1)*gb_team_2_second_pistol['count']

total_pistols = (gb_team_1_first_pistol['count']+gb_team_1_second_pistol['count']+gb_team_2_first_pistol['count']+gb_team_2_second_pistol['count'])

for major in majors[3:]:
    col = major['tournament']
    
    _filter = total_pistols > total_pistols.loc[:,col].quantile(0.3)
    
    gb.loc[:,col] = gb.loc[_filter,col]
    total_pistols.loc[:,col] = total_pistols.loc[_filter,col]
    
    gb.dropna(inplace=True)
    total_pistols.dropna(inplace=True)

mean_pistols = pd.DataFrame(gb/total_pistols)
mean_pistols.dropna(inplace=True)
mean_pistols.sort_values(['side','time_period',0],ascending=[1,1,0],inplace=True)

In [39]:
def get_rankings_pistols_side(side):
    ranking_pistols_side = pd.DataFrame(index=range(1,8))
    ranking_pistols_side.index.name = 'ranking'

    for major in majors[3:]:
        col = major['tournament']
        ranking_pistols_side[col] = mean_pistols.loc[side,col][0][:7].index
    
    return ranking_pistols_side

In [40]:
print('\nRankings Pistols CT-side:')
display(get_rankings_pistols_side('ct'))
print('\nRankings Pistols T-side:')
display(get_rankings_pistols_side('t'))


Rankings Pistols CT-side:


,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018,08. Katowice 2019,09. Berlin 2019
ranking,,,,,,
1,FaZe,FaZe,OpTic,Virtus.pro,FaZe,Heroic
2,CLG,North,BIG,Astralis,G2,BIG
3,Virtus.pro,Cloud9,Astralis,MIBR,Renegades,Liquid
4,SK,Virtus.pro,North,OpTic,MIBR,CR4ZY
5,Envy,OpTic,NRG,Renegades,OpTic,AVANGAR
6,Cloud9,HellRaisers,Space Soldiers,Complexity,Astralis,Astralis
7,North,Space Soldiers,Cloud9,Liquid,FURIA,ENCE



Rankings Pistols T-side:


,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018,08. Katowice 2019,09. Berlin 2019
ranking,,,,,,
1,FaZe,mousesports,Heroic,Liquid,Valiance,BIG
2,Cloud9,Renegades,Imperial,Astralis,Natus Vincere,100 Thieves
3,CLG,OpTic,NRG,G2,Astralis,fnatic
4,mousesports,HellRaisers,mousesports,Virtus.pro,fnatic,FaZe
5,SK,SK,Space Soldiers,NRG,Windigo,Sprout
6,North,Cloud9,GODSENT,FaZe,Liquid,Renegades
7,Immortals,Gambit,Astralis,Cloud9,Vitality,Evil Geniuses


# Players DataFrame

## Players ranking by map

In [41]:
players_df.head()

,date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,map_1,map_2,map_3,kills,assists,deaths,hs,flash_assists,kast,kddiff,adr,fkdiff,rating,m1_kills,m1_assists,m1_deaths,m1_hs,m1_flash_assists,m1_kast,m1_kddiff,m1_adr,m1_fkdiff,m1_rating,m2_kills,m2_assists,m2_deaths,m2_hs,m2_flash_assists,m2_kast,m2_kddiff,m2_adr,m2_fkdiff,m2_rating,m3_kills,m3_assists,m3_deaths,m3_hs,m3_flash_assists,m3_kast,m3_kddiff,...,m3_rating,kills_ct,deaths_ct,kddiff_ct,adr_ct,kast_ct,rating_ct,kills_t,deaths_t,kddiff_t,adr_t,kast_t,rating_t,m1_kills_ct,m1_deaths_ct,m1_kddiff_ct,m1_adr_ct,m1_kast_ct,m1_rating_ct,m1_kills_t,m1_deaths_t,m1_kddiff_t,m1_adr_t,m1_kast_t,m1_rating_t,m2_kills_ct,m2_deaths_ct,m2_kddiff_ct,m2_adr_ct,m2_kast_ct,m2_rating_ct,m2_kills_t,m2_deaths_t,m2_kddiff_t,m2_adr_t,m2_kast_t,m2_rating_t,m3_kills_ct,m3_deaths_ct,m3_kddiff_ct,m3_adr_ct,m3_kast_ct,m3_rating_ct,m3_kills_t,m3_deaths_t,m3_kddiff_t,m3_adr_t,m3_kast_t,m3_rating_t,time_period
0,2020-02-26,Brehze,Evil Geniuses,Liquid,United States,9136,2339385,4901,IEM Katowice 2020,3,Overpass,Nuke,Inferno,57,14,61,29,0.0,71.1,-4,79.9,0,1.04,11,3,18,5,0.0,65.2,-7,60.8,-1,0.70,30.0,7.0,24.0,16.0,0.0,73.5,6.0,99.2,6.0,1.38,16.0,4.0,19.0,8.0,0.0,73.1,-3.0,...,0.91,34.0,30.0,4.0,81.6,79.2,1.10,23.0,31.0,-8.0,77.5,60.0,0.97,8.0,10.0,-2.0,76.3,73.3,0.90,3.0,8.0,-5.0,31.9,50.0,0.34,17.0,10.0,7.0,93.7,83.3,1.41,13.0,14.0,-1.0,105.3,62.5,1.35,9.0,10.0,-1.0,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89,09. Berlin 2019
1,2020-02-26,Brehze,Evil Geniuses,Liquid,United States,9136,2339385,4901,IEM Katowice 2020,3,Overpass,Nuke,Inferno,57,14,61,29,0.0,71.1,-4,79.9,0,1.04,11,3,18,5,0.0,65.2,-7,60.8,-1,0.70,30.0,7.0,24.0,16.0,0.0,73.5,6.0,99.2,6.0,1.38,16.0,4.0,19.0,8.0,0.0,73.1,-3.0,...,0.91,34.0,30.0,4.0,81.6,79.2,1.10,23.0,31.0,-8.0,77.5,60.0,0.97,8.0,10.0,-2.0,76.3,73.3,0.90,3.0,8.0,-5.0,31.9,50.0,0.34,17.0,10.0,7.0,93.7,83.3,1.41,13.0,14.0,-1.0,105.3,62.5,1.35,9.0,10.0,-1.0,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89,09. Berlin 2019
2,2020-02-26,Brehze,Evil Geniuses,Liquid,United States,9136,2339385,4901,IEM Katowice 2020,3,Overpass,Nuke,Inferno,57,14,61,29,0.0,71.1,-4,79.9,0,1.04,11,3,18,5,0.0,65.2,-7,60.8,-1,0.70,30.0,7.0,24.0,16.0,0.0,73.5,6.0,99.2,6.0,1.38,16.0,4.0,19.0,8.0,0.0,73.1,-3.0,...,0.91,34.0,30.0,4.0,81.6,79.2,1.10,23.0,31.0,-8.0,77.5,60.0,0.97,8.0,10.0,-2.0,76.3,73.3,0.90,3.0,8.0,-5.0,31.9,50.0,0.34,17.0,10.0,7.0,93.7,83.3,1.41,13.0,14.0,-1.0,105.3,62.5,1.35,9.0,10.0,-1.0,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89,09. Berlin 2019
3,2020-02-26,CeRq,Evil Geniuses,Liquid,Bulgaria,11219,2339385,4901,IEM Katowice 2020,3,Overpass,Nuke,Inferno,54,10,54,18,4.0,65.1,0,71.7,2,0.98,11,2,17,4,2.0,60.9,-6,68.9,-1,0.75,26.0,6.0,19.0,8.0,1.0,76.5,7.0,80.1,3.0,1.24,17.0,2.0,18.0,6.0,1.0,53.8,-1.0,...,0.87,37.0,25.0,12.0,77.4,72.9,1.16,17.0,29.0,-12.0,63.9,54.3,0.73,9.0,9.0,0.0,72.3,73.3,0.88,2.0,8.0,-6.0,62.4,37.5,0.50,15.0,6.0,9.0,79.8,88.9,1.45,11.0,13.0,-2.0,80.5,62.5,1.00,13.0,10.0,3.0,79.5,53.3,1.12,4.0,8.0,-4.0,40.7,54.5,0.53,09. Berlin 2019
4,2020-02-26,CeRq,Evil Geniuses,Liquid,Bulgaria,11219,2339385,4901,IEM Katowice 2020,3,Overpass,Nuke,Inferno,54,10,54,18,4.0,65.1,0,71.7,2,0.98,11,2,17,4,2.0,60.9,-6,68.9,-1,0.75,26.0,6.0,19.0,8.0,1.0,76.5,7.0,80.1,3.0,1.24,17.0,2.0,18.0,6.0,1.0,53.8,-1.0,...,0.87,37.0,25.0,12.0,77.4,72.9,1.16,17.0,29.0,-12.0,63.9,54.3,0.73,9.0,9.0,0.0,72.3,73.3,0.88,2.0,8.0,-6.0,62.4,37.5,0.50,15.0,6.0,9.0,79.8,88.9,1.45,11.0,13.0,-2.0,80.5,62.5,1.00,13.0,10.0,3.0,79.5,53.3,1.12,4.0,8.0,-4.0,40.7,54.5,0.53,09. Berlin 2019


In [42]:
all_maps_columns = ['date','time_period','country','player_name','team','opponent','player_id',
                    'match_id','event_id','event_name','best_of']
each_map_columns = ['kills','assists','deaths','hs','flash_assists','kast','kddiff','adr','fkdiff','rating']

In [43]:
map1_columns = ['map_1']+['m1_'+ x for x in each_map_columns]
map2_columns = ['map_2']+['m2_'+ x for x in each_map_columns]
map3_columns = ['map_3']+['m3_'+ x for x in each_map_columns]

In [44]:
out_columns = all_maps_columns+['_map']+each_map_columns

players_df_by_map_columns = pd.DataFrame(columns=out_columns)

In [45]:
#Countries that contribute the most to the professional scene by number of matches
players_df.groupby('country')['country'].count().sort_values(ascending=False)[:30]

country
Denmark                   13562
Sweden                    11031
United States              9216
France                     7116
Brazil                     6463
Poland                     5415
Russia                     4390
Ukraine                    3640
Canada                     3295
Germany                    3050
Finland                    2358
Australia                  2020
Kazakhstan                 1917
Turkey                     1801
Slovakia                   1791
Norway                     1737
Bulgaria                   1402
China                       856
Bosnia and Herzegovina      852
Belgium                     851
Netherlands                 823
Estonia                     750
Serbia                      619
Czech Republic              587
United Kingdom              585
Spain                       551
Hungary                     417
Lithuania                   307
Jordan                      299
Indonesia                   212
Name: country, dtype: int64

In [46]:
curr_map = {}
curr_map[0] = players_df[(all_maps_columns+map1_columns)]
curr_map[1] = players_df[(all_maps_columns+map2_columns)]
curr_map[2] = players_df[(all_maps_columns+map3_columns)]

curr_map[0].columns = out_columns
curr_map[1].columns = out_columns
curr_map[2].columns = out_columns

all_maps = pd.concat(   (   pd.concat(   (curr_map[0],curr_map[1])    ), curr_map[2]   )   )

In [47]:
gb2 = all_maps.groupby(['time_period','player_id','_map'])
threshold_maps_played = 7
all_maps2 = gb2.filter(lambda x:x.player_name.count()>threshold_maps_played)
all_maps2.head()

,date,time_period,country,player_name,team,opponent,player_id,match_id,event_id,event_name,best_of,_map,kills,assists,deaths,hs,flash_assists,kast,kddiff,adr,fkdiff,rating
0,2020-02-26,09. Berlin 2019,United States,Brehze,Evil Geniuses,Liquid,9136,2339385,4901,IEM Katowice 2020,3,Overpass,11.0,3.0,18.0,5.0,0.0,65.2,-7.0,60.8,-1.0,0.70
1,2020-02-26,09. Berlin 2019,United States,Brehze,Evil Geniuses,Liquid,9136,2339385,4901,IEM Katowice 2020,3,Overpass,11.0,3.0,18.0,5.0,0.0,65.2,-7.0,60.8,-1.0,0.70
2,2020-02-26,09. Berlin 2019,United States,Brehze,Evil Geniuses,Liquid,9136,2339385,4901,IEM Katowice 2020,3,Overpass,11.0,3.0,18.0,5.0,0.0,65.2,-7.0,60.8,-1.0,0.70
3,2020-02-26,09. Berlin 2019,Bulgaria,CeRq,Evil Geniuses,Liquid,11219,2339385,4901,IEM Katowice 2020,3,Overpass,11.0,2.0,17.0,4.0,2.0,60.9,-6.0,68.9,-1.0,0.75
4,2020-02-26,09. Berlin 2019,Bulgaria,CeRq,Evil Geniuses,Liquid,11219,2339385,4901,IEM Katowice 2020,3,Overpass,11.0,2.0,17.0,4.0,2.0,60.9,-6.0,68.9,-1.0,0.75


In [48]:
gb = all_maps2.groupby(['time_period','_map','player_name'],sort=False)['rating','kddiff'].mean()
rankings = gb.sort_values(['time_period','_map','rating'],ascending=[1,1,0])

In [49]:
rankings_players = {}
for _map in maps:
    rankings_players[_map] = pd.DataFrame(index=range(1,21))
    rankings_players[_map].index.name = 'ranking'
    
    for major in majors:
        col = major['tournament']
        try:
            rankings_players[_map][col] = rankings.loc[col,_map].rating[:20].index
        except:
            pass

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: PerformanceWarning:

indexing past lexsort depth may impact performance.



Best players on each map accordding to their average Rating 2.0:

In [50]:
for _map in maps:
    print('\n'+_map+':')
    display(rankings_players[_map])


Cache:


,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018,08. Katowice 2019
ranking,,,,,,,,
1,oskar,MICHU,HEN1,innocent,boltz,EspiranTo,Jame,NBK-
2,WorldEdit,ScreaM,olofmeister,Calyx,NAF,NiKo,NAF,ZywOo
3,olofmeister,felps,ANGE1,fer,XANTARES,nexa,Twistzz,RpK
4,NiKo,shox,Zero,HS,coldzera,woxic,f0rest,KRIMZ
5,coldzera,innocent,Kjaerbye,XANTARES,oskar,device,tabseN,JW
6,kennyS,MINISE,mixwell,suNny,FalleN,ISSAA,XANTARES,Brollan
7,JW,NiKo,coldzera,ANGE1,nex,Ethan,flusha,AmaNEk
8,NBK-,oskar,Furlan,NiKo,GruBy,GuardiaN,EspiranTo,Jame
9,Shara,boltz,WorldEdit,STYKO,chelo,NAF,qikert,apEX



Cobblestone:


,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018
ranking,,,,,,
1,Zero,kennyS,Hobbit,coldzera,paz,paz
2,felps,apEX,Dima,Calyx,Furlan,Calyx
3,Snax,AZR,shox,XANTARES,XANTARES,XANTARES
4,queztone,ScreaM,XANTARES,fer,snatchie,woxic
5,oskar,olofmeister,tarik,Twistzz,f0rest,NAF
6,f0rest,oskar,NiKo,MICHU,REZ,KRIMZ
7,MINISE,nitr0,mou,tabseN,s1mple,fer
8,mouz,MODDII,spaze,bodyy,NAF,ngiN
9,Happy,EliGE,Dosia,Skadoodle,GruBy,Twistzz



Dust2:


...

,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018,08. Katowice 2019,09. Berlin 2019
ranking,,,,,,,,,
1,flusha,shox,s1mple,ShahZaM,coldzera,cromen,apEX,TeSeS,yuurih
2,Magisk,Stewie2K,Magisk,tabseN,fer,s1mple,tabseN,NiKo,roeJ
3,dupreeh,oskar,flamie,AdreN,s1mple,CeRq,ZywOo,REZ,rain
4,NAF,dupreeh,NiKo,SicK,rain,Ethan,s1mple,ZywOo,ropz
5,koosta,fer,bubble,bondik,oskar,Brollan,AmaNEk,EliGE,acoR
6,MODDII,ScreaM,v1c7oR,XANTARES,ropz,EspiranTo,Ethan,Jame,coldzera
7,s1mple,shroud,freddieb,fer,Calyx,FugLy,Aleksib,blameF,Brollan
8,k0nfig,device,NAF,fnx,Zero,device,allu,s1mple,Magisk
9,kennyS,Magisk,RUSH,Lekr0,NiKo,REZ,Kjaerbye,Boombl4,KSCERATO



Train:


,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018,08. Katowice 2019,09. Berlin 2019
ranking,,,,,,,,,
1,rallen,f0rest,FalleN,suNny,GruBy,Aerial,s1mple,s1mple,s1mple
2,facecrack,rain,coldzera,GeT_RiGhT,s1mple,s1mple,device,electronic,electronic
3,Furlan,bubble,bubble,draken,device,XANTARES,Boombl4,CeRq,Brollan
4,FalleN,SicK,fer,s1mple,k0nfig,allu,Ethan,nexa,mouz
5,repo,device,flusha,XANTARES,XANTARES,nexa,NiKo,Xyp9x,almazer
6,WorldEdit,coldzera,AdreN,Hobbit,fer,JUGi,jks,aizy,xsepower
7,fer,mou,device,device,electronic,woxic,Brehze,Brollan,HEN1
8,jdm64,flamie,Lekr0,FalleN,RpK,device,woxic,Brehze,innocent
9,JW,Magisk,flamie,AdreN,f0rest,Ethan,valde,NiKo,nawwk



Vertigo:


,08. Katowice 2019,09. Berlin 2019
ranking,,
1,k0nfig,Brollan
2,jkaem,STYKO
3,EliGE,huNter-
4,device,zehN
5,huNter-,jks
6,nexa,device
7,Xyp9x,Golden
8,MSL,Liazz
9,niko,STOMP


## Players ranking all maps

In [51]:
ranking_players_df = pd.DataFrame()
ranking_players_df['player'] = players_df.player_name.unique()
ranking_players_df.set_index('player',inplace=True)

for major in majors:
    col = major['tournament']
    ranking_players_df[col] = 0
    
for _map in maps:
    for col in rankings_players[_map].columns:
        for i in range(1,21):
            ranking_players_df.loc[rankings_players[_map][col][i],col] += 21-i
            
rankings_players_again = {}

rankings_players_again = pd.DataFrame(index=range(1,21))
rankings_players_again.index.name = 'ranking'

for major in majors:
    col = major['tournament']
    rankings_players_again[col] = ranking_players_df[col].sort_values(ascending=False)[:20].index

Below it is shown the player rankings if it was only determined by the rating 2.0 across all maps. It can be considered as the ranking of most versatile fraggers. 

In [52]:
rankings_players_again

,01. Cluj-Napoca 2015,02. Columbus 2016,03. Cologne 2016,04. Atlanta 2017,05. Krakow 2017,06. Boston 2018,07. London 2018,08. Katowice 2019,09. Berlin 2019
ranking,,,,,,,,,
1,oskar,oskar,NiKo,XANTARES,s1mple,device,s1mple,s1mple,ZywOo
2,NiKo,ScreaM,bubble,fer,XANTARES,XANTARES,ZywOo,ZywOo,device
3,olofmeister,device,coldzera,Calyx,NiKo,s1mple,Jame,electronic,s1mple
4,WorldEdit,coldzera,device,NiKo,oskar,woxic,woxic,Jame,Brollan
5,GuardiaN,shox,flusha,tabseN,rain,Ethan,mertz,EliGE,ropz
6,kennyS,rain,ANGE1,coldzera,coldzera,nexa,device,device,KSCERATO
7,NAF,Magisk,dennis,AdreN,GruBy,NiKo,tabseN,nexa,huNter-
8,HEN1,flamie,HEN1,Twistzz,NAF,NAF,valde,NiKo,tabseN
9,JW,flusha,Magisk,ShahZaM,FalleN,Magisk,Twistzz,blameF,syrsoN


## K/D Graph

In [53]:
results_df_rank_part_1 = results_df[['match_id','team_1','rank_1']].rename(columns={'team_1':'team','rank_1':'team_rank'})
results_df_rank_part_2 = results_df[['match_id','team_2','rank_2']].rename(columns={'team_2':'team','rank_2':'team_rank'})
results_df_rank = pd.concat((results_df_rank_part_1,results_df_rank_part_2))

In [54]:
all_maps3 = all_maps2.merge(results_df_rank,'left',on=['match_id','team'])

In [55]:
players_series = all_maps3.groupby('player_name').team_rank.min()
players_list = list(players_series[players_series<=3].index)

In [56]:
gb = all_maps3.groupby(['time_period','player_name','country'])['kills','deaths','team_rank']
gb = gb.mean()[gb.count()['kills']>100]
gb = gb[gb.index.get_level_values(1).isin(players_list)]

gb['kills'] = gb['kills'].round(1)
gb['deaths'] = gb['deaths'].round(1)
gb['team_rank'] = gb['team_rank'].round(0).astype('int')
gb.reset_index(inplace=True)

In [57]:
gb['region'] = ''
gb.loc[(gb['country']=='Ukraine') | (gb['country']=='Russia') | (gb['country']=='Kazakhstan'),'region'] = 'CIS'
gb.loc[(gb['country']=='Brazil'),'region'] = 'Brazil'
gb.loc[(gb['country']=='France') | (gb['country']=='Belgium'),'region'] = 'France/Belgium'
gb.loc[(gb['country']=='United States') | (gb['country']=='Canada'),'region'] = 'North America'
gb.loc[(gb['country']=='Denmark'),'region'] = 'Denmark'
gb.loc[(gb['country']=='Sweden'),'region'] = 'Sweden'
gb.loc[(gb['country']=='Poland'),'region'] = 'Poland'
gb.loc[gb.country.isin(['Netherlands','Slovakia','Bosnia and Herzegovina',
                        'Norway','Czech Republic','Spain','Estonia','United Kingdom','Portugal','Turkey','Bulgaria', 'Finland']),'region'] = 'Rest of Europe'

gb = gb.sort_values(['time_period','region'])

In [58]:
gb['size'] = (100/(gb['team_rank']+2)).round(1)

In [59]:
fig = px.scatter(gb, x="deaths", y="kills", animation_frame="time_period", animation_group="player_name",
           size="size", color="region", hover_name="player_name", hover_data=["team_rank"],
                 range_x=[14,24],range_y=[14,24]
                )
#.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 1300
fig.update_layout(xaxis_title='Deaths', yaxis_title = 'Kills')
fig.add_shape(type="line", x0=14, y0=14, x1=24, y1=24, line=dict(width=4, dash="dot"))
fig.update_shapes(dict(xref='x', yref='y'))
fig.show()

# Picks DataFrame

In [60]:
picks_df.head()

,date,team_1,team_2,inverted_teams,match_id,event_id,best_of,system,t1_removed_1,t1_removed_2,t1_removed_3,t2_removed_1,t2_removed_2,t2_removed_3,t1_picked_1,t2_picked_1,left_over,time_period
3,2020-03-17,Complexity,forZe,1,2340279,5226,3,123412,Inferno,Nuke,0.0,Overpass,Vertigo,0.0,Dust2,Train,Mirage,09. Berlin 2019
5,2020-03-17,BIG,fnatic,1,2340397,5226,3,123412,Train,Dust2,0.0,Nuke,Vertigo,0.0,Inferno,Overpass,Mirage,09. Berlin 2019
15,2020-03-17,North,Natus Vincere,1,2340396,5226,3,123412,Train,Inferno,0.0,Vertigo,Mirage,0.0,Nuke,Dust2,Overpass,09. Berlin 2019
23,2020-03-16,Spirit,ENCE,1,2340391,5226,3,123412,Overpass,Dust2,0.0,Vertigo,Nuke,0.0,Inferno,Train,Mirage,09. Berlin 2019
30,2020-03-16,GODSENT,Vitality,1,2340063,5226,3,123412,Mirage,Overpass,0.0,Train,Inferno,0.0,Vertigo,Nuke,Dust2,09. Berlin 2019


In [61]:
gb = picks_df.groupby('system').system.count()
gb = gb[gb>10]
gb

system
112221     208
121212     354
121234      81
122112     406
123412    1546
123421     159
Name: system, dtype: int64

The order 123412 (t1_remove, t2_remove, t1_pick, t2_pick, t1_remove, t2_remove, left_over) is by far the most common map picking system for bo3 matches.
For bo1 matches, there is a three-way split between the systems 122112, 112221 and 121212.

# Further exploration and predictions

If you think these insights are useful, I might update the dataset with data from 2014 and 2015.

What insights can you create from this data? Can you predict the probability of each:
 - team winning a map?
 - map being vetoed or selected?
 - team winning the match, combining predictions from map winners and map picks?
 
 Give it a try!
